In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import os
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

from get_stock_data import GetStockData

In [3]:
username = "mth989722"
password = "SystTrade2022"
conn = wrds.Connection(wrds_username='mth989722')

Loading library list...
Done


In [7]:
# for ticker_file in os.listdir("./stock_data/"):
#     ticker = ticker_file[:-4]
#     data = GetStockData._get_stock_data(conn=conn, ticker=ticker, startdate='01/01/2018', enddate='12/31/2021')
#     data.to_csv(f"./stock_data/{ticker}.csv")

In [3]:
# conn.describe_table(library='crsp', table='dsf')
# conn.describe_table(library='crsp', table='msenames')

### Get S&P 500 Constituent Stock List

In [3]:
import bs4 as bs
import requests
import yfinance as yf
import datetime
from sec_edgar_downloader import Downloader

In [5]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [6]:
sp500_tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    sp500_tickers.append(ticker)
sp500_tickers = [s.replace("\n", "") for s in sp500_tickers]

### Download edgar/stock data for S&P500 stocks

In [1]:
import time
from tqdm import tqdm
from multiprocessing import Pool

if not os.path.exists("./stock_data"):
    os.mkdir("./stock_data")

error_tickers = []

def download_edgar_data(ticker, cnt, error_tickers):
    try:
        dl = Downloader("./")
        if not os.path.exists(f'./stock_data/{ticker}.csv'):
            cnt[0] += 1
            data = GetStockData._get_stock_data(conn=conn, ticker=ticker, startdate='01/01/2018', enddate='12/31/2021')
            if not data is None:
                print('Start downloading ', ticker)
                dl.get("4", ticker, after="2018-01-01", before="2021-12-31")
                data.to_csv(f"./stock_data/{ticker}.csv")
            if cnt[0] % 5 == 0:
                time.sleep(120)
        return None
    except Exception as e: 
        print("Error ticker: ", ticker, ", error: ", e)
        error_tickers.append(ticker)
        return None


In [ ]:
cnt = 0
while True:
    conn = wrds.Connection(wrds_username='mth989722')
    for ticker in tqdm(sp500_tickers):
        download_edgar_data(ticker, [cnt], error_tickers)
    time.sleep(1200)

In [4]:
download_edgar_data("AMAT", [0], [])